## BigCommerce  Attributes

In [1]:
import pandas as pd
from connect_training import wb_Db
from string_cleaner import show_dictionary
from string_cleaner import showDF
import csv
from bs4 import BeautifulSoup
db = wb_Db()

In [2]:
def pullBcCustomFields():
    query = f"""
        select "Product ID"
        , "Product Code/SKU"
        ,"Category String"
                , "Product URL"
        ,"Product Custom Fields" 
        from bc_items_all_cols biac 
        """
    list_ = db.runQuery(query)
    columns = ["Product ID"
        , "Product Code/SKU"
        ,"Category String"
                       , "Product URL"
        ,"Product Custom Fields"]
    return list_, columns



def convertBCAttributeToDict(record, see_set = False):
    """
    Input: BigCommerce Product Customs Field in the below format to a collection on key-value pairs (dictionary)
    STRING ----"ATTRIBUTE=VALUE"; "ATTRIBUTE=VALUE"; "ATTRIBUTE=VALUE"; "ATTRIBUTE=VALUE"; ...
    Ouput Dictionary
    """
    dict_ = {}
    set_ = set()
    for pair in record.split(';'):
        pair = [i for i in pair.replace('"','').split('=')]
        if len(pair)>1:
            dict_[pair[0]] = pair[-1]
            set_.add(pair[0])
            
    if see_set:
        return dict_, set_
    return dict_




def create_Attribute_DataFrame():
    """
    Input: SQL select query from 
    
    """
    list_, columns = pullBcCustomFields()
    df = pd.DataFrame(data= list_, columns= columns)
    records = [convertBCAttributeToDict(record[-1]) for record in list_]
    
    df_ = pd.DataFrame.from_records(records)
    for i in df_.columns:
        df[i] = df_[i]
     
    return df 

def removeEmptyFields(a):
    """
    Entering dataframe
    Output dataframe without empty fields.
    """
    length = len(a)
    for col in a.columns:
        if length == a[col].isna().sum():
            a = a.drop(columns=[col])
    return a

In [3]:
bc = create_Attribute_DataFrame()
# df= df.drop(columns=['Product Custom Fields'])
# df.to_csv('upload_files/update_table.csv', index=False)
showDF(bc)

Number of Records - 191805


,Product ID,Product Code/SKU,Category String,Product URL,Product Custom Fields,Number of Teeth,Dual-Sided,Belt Series,Belt Pitch Length (mm),Length (in),...,Compatible Cone,Cup Width Inch,Cup Width MM,Single or Double Cup,Series,Inner Ring Width Inch,Inner Ring Width MM,Outer Ring Width Inch,Outer Ring Width MM,Cylindrical or Spherical OD
0,48734,72XL075 - D&D GLOBAL,Transmission Belts/Synchronous Belts,/transmission-belts/synchronous-belts/72xl075-...,"""Number of Teeth=36"";Dual-Sided=No;""Belt Serie...",36,No,XL,5.08,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create Hyperlink
bc['Links'] = [f'https://3bgsupply.com{i[2]}' for i in bc[['Product Code/SKU', "Product URL"]].itertuples()]

In [5]:
with open ("check.html", "r") as myfile:
    html_doc='\n'.join(myfile.readlines())
soup = BeautifulSoup(html_doc, 'html.parser')


In [6]:
list_ = []
for tag in soup.find_all('input'):
    check = '' if tag.get('name')==None else tag.get('name')
    if '[label]' in check:
        list_.append(tag.get('value'))
        
df = pd.DataFrame(data =list_, columns=['label'] )

In [7]:
len(list_)

626

In [8]:
df['Field'] = [i.string for i in soup.find_all('strong')]
    
df    

,label,Field
0,abma_precision_rating,abma_precision_rating
1,adapter_part_number_inch,adapter_part_number_inch
2,adapter_part_number_mm,adapter_part_number_mm
3,adapter_sleeve,adapter_sleeve
4,assembly_components,assembly_components
...,...,...
621,withdrawal_nut,withdrawal_nut
622,withdrawal_sleeve,withdrawal_sleeve
623,w_dim_inch,w_dim_inch
624,x_dim_inch,x_dim_inch


In [9]:
[i for i in bc.columns if i not in df['label'].unique()]

['Product ID',
 'Product Code/SKU',
 'Category String',
 'Product URL',
 'Product Custom Fields',
 'Belt Pitch Length (mm)',
 'Height (In)',
 'Width (In)',
 'Belt Pitch Length (in)',
 'Pitch (mm)',
 'Power Factor',
 'Full-Load Torque',
 'Overall Height Inch',
 'Overall Height MM',
 'Inverter Rated or Duty',
 '24',
 '11',
 'Length - Inch',
 'Width Inch',
 'Height Inch',
 'Bolt Center to Center MM',
 'Max Nom Bolt Ctr to Ctr Inch',
 'Max Nom Bolt Ctr to Ctr MM',
 'Expansion / Non-Expansion',
 'Min Nom Bolt Ctr to Ctr Inch',
 'Min Nom Bolt Ctr to Ctr MM',
 'Base to Center Height MM',
 'Length Across Bore MM',
 'Nom Bolt Circle Dia Round Inch',
 'Nom Bolt Circle Dia Round MM',
 'Cartridge Pilot Depth Inch',
 'Cartridge Pilot Depth MM',
 'Cartridge Pilot Diameter Inch',
 'Cartridge Pilot Diameter MM',
 'Width MM',
 'Mounting Method',
 'Inner Race Width Inch',
 'Inner Race Width MM',
 'Bore (In)',
 'Bore (Mm)',
 'Outside Diameter (In)',
 'Outside Diameter (Mm)',
 'Outer Race Width (In)',
 'O

In [10]:
func = lambda x: (x not in bc.columns) & ('_' not in x)
df['Match BigCommerce'] = df['label'].apply(lambda x: x in bc.columns)
check = df['label'].apply(lambda x: '_' in x)
df.loc[check, 'Remove'] = True
df

,label,Field,Match BigCommerce,Remove
0,abma_precision_rating,abma_precision_rating,False,True
1,adapter_part_number_inch,adapter_part_number_inch,False,True
2,adapter_part_number_mm,adapter_part_number_mm,False,True
3,adapter_sleeve,adapter_sleeve,False,True
4,assembly_components,assembly_components,False,True
...,...,...,...,...
621,withdrawal_nut,withdrawal_nut,False,True
622,withdrawal_sleeve,withdrawal_sleeve,False,True
623,w_dim_inch,w_dim_inch,False,True
624,x_dim_inch,x_dim_inch,False,True


In [11]:
df.to_csv('searchspring.csv', index=False)

In [12]:
bc_columns = """11
24
A Dim Inch
ABMA Precision Rating
Adapter Part Number Inch
Adapter Part Number MM
Adapter Sleeve
Assembly Components
Assembly Number
Assembly Width Inch
Assembly Width MM
B Dim Inch
Ball Material
Ball Width Inch
Ball Width MM
Banded
Base to Center Height Inch
Base to Center Height MM
Bearing Arrangement
Bearing Insert Part Number
Bearing Locking Style
Bearing Material
Bearing Profile
Belt Pitch Length (in)
Belt Pitch Length (mm)
Belt Series
Belt Width Inch
Belt Width MM
Bolt Center to Center Inch
Bolt Center to Center MM
Bolt Circle Inch
Bore (In)
Bore (Mm)
Bore / Inner Diameter Inch
Bore / Inner Diameter MM
Bore 2 Inch
Bore 2 MM
Bore Outer Diameter Inch
Bore Outer Diameter MM
Bore Profile
Bore Type
Breakdown Torque
Bushing Style
C Dim Inch
Cage Material
Cartridge Housing Width Inch
Cartridge Housing Width MM
Cartridge Pilot Depth Inch
Cartridge Pilot Depth MM
Cartridge Pilot Diameter Inch
Cartridge Pilot Diameter MM
Cogged
Compatible Cone
Compatible Cup
Compatible Take-Up Frame
Cone Width Inch
Cone Width MM
Contact Angle
Contact Angle 2
corrected_weight
Cup Width Inch
Cup Width MM
Cylindrical or Spherical OD
D Dim Inch
D Large
D Small
Diametral Pitch
Direction
Down Thrust
Drive End Bottom Bearing
Drive End Upper Bearing
Dual-Sided
Dynamic Load Capacity
Dynamic Load Capacity lbf
Dynamic Load Capacity N
E Dim Inch
Enclosure
Enclosure Type
End Play
Expansion / Non-Expansion
F Dim Inch
Face Width Inch
Face Width MM
Fillet Radius
Finish
Finish / Coating
Flange Diameter Inch
Flange Diameter MM
Flange Material
Flange Mount Style
Flange Outside Diameter Inch
Flange Outside Diameter MM
Flanges
Flush Ground
Frame
Frequency Hz
Full Load Amps at Rated Voltage
Full Load Efficiency Percentage
Full Load RPM
Full-Load Torque
G Dim Inch
Garter Spring Present
H Dim Inch
Height (In)
Height (mm)
Height Inch
Height MM
Hex Socket
Horse Power
Housing Material
Housing Outside Diameter Inch
Housing Outside Diameter MM
Housing Overall Length Inch
Housing Overall Length MM
Housing Style
Housing Width Inch
Housing Width MM
Hub Diameter Inch
Hub Diameter MM
I Dim Inch
ID Inch
In Or Metric
Inner Race Profile
Inner Race Width Inch
Inner Race Width MM
Inner Ring Width Inch
Inner Ring Width MM
Insulation Class
Intended Pulley Angle
Internal Fit / Preload
Internal Special Features
Inverter Rated or Duty
Length - Inch
Length (in)
Length (mm)
Length Across Bore Inch
Length Across Bore MM
Length Inch
Length MM
Length Thru Bore (In)
Length Thru Bore (Mm)
Liner Material
Lip Style
Locked Rotor Amps at Rated Voltage
Locked-Rotor Torque
M Dim Inch
Material
Max Nom Bolt Ctr to Ctr Inch
Max Nom Bolt Ctr to Ctr MM
Maximum Bore Inch
Min Nom Bolt Ctr to Ctr Inch
Min Nom Bolt Ctr to Ctr MM
Min Usable Length Inch
Min Usable Length MM
Minimum Bore Inch
Mounting Arrangement
Mounting Bolts Inch
Mounting Bolts MM
Mounting Method
NEMA Code
NEMA Design
Nom Bolt Circle Dia Round Inch
Nom Bolt Circle Dia Round MM
Nom Bolt Ctr To Ctr (In)
Nom Bolt Ctr To Ctr (Mm)
Nom. Bolt Circle Dia. Round Inch
Note
Number of Bands
Number of Bearings
Number of Mounting Holes
Number of Ribs
Number of Rows of Balls
Number of Rows of Rollers
Number of Tapered Rows
Number of Teeth
Operating Temperature Range
Opposite Drive End Bottom Bearing
Opposite Drive End Upper Bearing
Outer Case Material
Outer Diameter Inch
Outer Diameter MM
Outer Member Material
Outer Race Width (In)
Outer Race Width (Mm)
Outer Race Width Inch
Outer Race Width MM
Outer Ring Width Inch
Outer Ring Width MM
Outside Diameter (In)
Outside Diameter (Mm)
Outside Diameter Design
Outside Diameter Inch
Outside Diameter MM
Overall Height Inch
Overall Height MM
Overall Length Inch
Overall Length MM
Overall Thickness Inch
Overall Width Inch
Overall Width MM
Part Winding Start 230V
Part Winding Start 460V
Phase
Piloted
Pitch (mm)
Pitch Diameter Inch
Pitch Diameter MM
Power Factor
Precision Class
Pressure Angle
PSI
Pulley Belt Width Inch
Pulley Belt Width MM
Race Material
Raceway Style
Relubricatable
Retainer
Roller Diameter Inch
Roller Diameter MM
Roller Width Inch
Roller Width MM
Rolling Element
RPM
Seal Type
Self Aligning
Separable
Series
Service Factor
Shaft Inner Diameter Inch
Shaft Inner Diameter MM
Single or Double Cup
Slot to Slot Distance Inch
Slot to Slot Distance MM
Slot Width Inch
Slot Width MM
Snap Ring
Snap Ring Groove
Split Housing
Static Load Capacity
Static Load Capacity lbf
Static Load Capacity N
Stud Diameter Inch
Stud Diameter MM
Stud Profile
Surface Profile
T Dim Inch
Teeth
Thickness Inch
Thread Direction
Thread Size
Thread Style
Timing Belt Pitch Inch
Timing Belt Pitch MM
Tooth Style
Torque Capacity Lb/Inch
Travel Length
Type
U Dim Inch
Upper Width (Per Section - in)
V Dim Inch
Voltage
W Dim Inch
weight
Weight (lbs)
Width (In)
Width (mm)
Width Inch
Width MM
Withdrawal Nut
Withdrawal Sleeve
X Dim Inch
Y Dim Inch""".split('\n')

In [13]:
bc_columns = [i.strip() for i in bc_columns]

In [14]:
for i in bc_columns:
    print(i in df['label'].unique())

False
False
False
False
False
False
True
True
True
True
True
False
True
False
False
True
True
False
True
True
True
True
True
False
False
True
True
True
True
False
True
False
False
True
True
False
False
True
True
True
True
True
True
False
True
True
True
False
False
False
False
True
True
True
True
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
False
False
True
True
True
True
False
True
True
True
True
False
True
True
True
True
True
True
True
True
False
False
True
False
False
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
False
False
False
True
True
True
False
False
False
True
True
True
False
False
False
False
False
True
True
True
True
False
True
False
False
True
False
False
True
True
True
False
True
False
False
True
True
False
False
False
False
True
False
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
False
True
Tru

In [15]:
list_1 = []
list_2 = []

for i in bc_columns:
    bc_= bc.loc[bc[i].isna()==False].sample(n=1)
    
    list_1.append(bc_['Product Code/SKU'].unique()[0])
#     list_1.append(i)
    list_2.append(bc_['Links'].unique()[0])

In [16]:
for i in list_1:
    print(i)

B6X23F - MACHINE GUARD
B8X19G - MACHINE GUARD
3030X2-3/16 - B & B MANUFACTURING
JM207049 - PEER BEARING COMPANY
23134-E1A-K-M-C3 - FAG BEARING
239/710 CAK/HA3C08W507 - SKF
2309-K-2RS-TVH-C3 - FAG BEARING
3382-90036 - TIMKEN
HM133436-90312 - TIMKEN
31328-X - FAG BEARING
81575-90125 - TIMKEN
1008X20MM - B & B MANUFACTURING
3MMV9316HX DUL - FAFNIR
TREL-4N - SEALMASTER
SAKAC 6 M - SKF
T711-902A1 - TIMKEN
ZAS2207 - REXNORD
PELB6827D8C - LINK-BELT
2MMC9112WI SUL - FAFNIR
LSE300BXHSAFQAATL - TIMKEN
SAF 22634X 5 13/16 - TORRINGTON
206KRP4 - PEER BEARING COMPANY
MUOA 2 - FAFNIR
484K4 - D&D GLOBAL
D1764-14M-170 - D&D GLOBAL
435-5M-09 - D&D GLOBAL
QD24-8M-30 - B & B MANUFACTURING
QD24-8M-30 - B & B MANUFACTURING
ASPP206 - NTN
SF-24T HS - SEALMASTER
SHX28MM - B & B MANUFACTURING
YET 207 107 - SKF
YET 211-203 - SKF
MEP2207F - REXNORD
1212-TVH - FAG BEARING
T20751-90010 - TIMKEN
T53250-903A2 - TIMKEN
5X16X6SC-BX - DICHTOMATIK AMERICAS
40X58X8SB-H - DICHTOMATIK AMERICAS
NN3020KC1NAP5 - NTN
4535X2-1/2

In [17]:
for i in list_2:
    print(i)

https://3bgsupply.com/machine-guards/standard-guards/standard-flanged-backplates/b6x23f-machine-guard
https://3bgsupply.com/machine-guards/standard-guards/standard-flanged-backplates/b8x19g-machine-guard
https://3bgsupply.com/bushings/tl-bushing/3030x2-3-16-b-b-manufacturing
https://3bgsupply.com/bearings/roller-bearing/tapered-roller-bearings/jm207049-peer-bearing-company
https://3bgsupply.com/bearings/roller-bearing/23134-e1a-k-m-c3-fag-bearing
https://3bgsupply.com/bearings/roller-bearing/239-710-cak-ha3c08w507-skf
https://3bgsupply.com/bearings/ball-bearing/2309-k-2rs-tvh-c3-fag-bearing
https://3bgsupply.com/bearings/roller-bearing/3382-90036-timken
https://3bgsupply.com/bearings/roller-bearing/hm133436-90312-timken
https://3bgsupply.com/bearings/roller-bearing/31328-x-fag-bearing
https://3bgsupply.com/bearings/roller-bearing/81575-90125-timken
https://3bgsupply.com/bushings/tl-bushing/1008x20mm-b-b-manufacturing
https://3bgsupply.com/bearings/ball-bearing/3mmv9316hx-dul-fafnir
htt

In [18]:
[i for i in removeEmptyFields(bc.loc[bc['Overall Height Inch'].isna()==False]).columns if 'Height' in i]

['Overall Height Inch', 'Overall Height MM']

In [19]:
[i for i in removeEmptyFields(bc.loc[bc['Overall Length Inch'].isna()==False]).columns if 'Length' in i]

['Overall Length Inch',
 'Overall Length MM',
 'Min Usable Length Inch',
 'Min Usable Length MM']

In [20]:
# Overall Width Inch
[i for i in removeEmptyFields(bc.loc[bc['Overall Width Inch'].isna()==False]).columns if 'Width' in i]

['Housing Width Inch',
 'Housing Width MM',
 'Overall Width Inch',
 'Overall Width MM',
 'Inner Ring Width Inch',
 'Inner Ring Width MM',
 'Outer Ring Width Inch',
 'Outer Ring Width MM']

In [21]:
# Roller Diameter Inch
[i for i in removeEmptyFields(bc.loc[bc['Roller Diameter Inch'].isna()==False]).columns if 'Inch' in i]

['Bore / Inner Diameter Inch',
 'Roller Diameter Inch',
 'Roller Width Inch',
 'Stud Diameter Inch']

In [22]:
# Timing Belt Pitch Inch
[i for i in removeEmptyFields(bc.loc[bc['X Dim Inch'].isna()==False]).columns if ' ' in i]

['Product ID',
 'Product Code/SKU',
 'Category String',
 'Product URL',
 'Product Custom Fields',
 'Bore / Inner Diameter Inch',
 'Bore / Inner Diameter MM',
 'Thread Size',
 'Bore Type',
 'Minimum Bore Inch',
 'Maximum Bore Inch',
 'Bushing Style',
 'B Dim Inch',
 'D Dim Inch',
 'E Dim Inch',
 'F Dim Inch',
 'I Dim Inch',
 'Torque Capacity Lb/Inch',
 'D Large',
 'D Small',
 'H Dim Inch',
 'T Dim Inch',
 'U Dim Inch',
 'V Dim Inch',
 'X Dim Inch',
 'Y Dim Inch',
 'W Dim Inch']

In [23]:
removeEmptyFields(bc.loc[bc['Mounting Arrangement'].isna()==False]).columns

Index(['Product ID', 'Product Code/SKU', 'Category String', 'Product URL',
       'Product Custom Fields', 'Width Inch', 'Bore / Inner Diameter Inch',
       'Bore / Inner Diameter MM', 'Rolling Element', 'Outside Diameter Inch',
       'Outside Diameter MM', 'Seal Type', 'Width MM', 'Number of Bearings',
       'Precision Class', 'Contact Angle', 'Cage Material', 'Flush Ground',
       'Contact Angle 2', 'Raceway Style', 'Ball Material',
       'Bearing Arrangement', 'Enclosure Type', 'Mounting Arrangement',
       'Internal Special Features', 'Links'],
      dtype='object')

In [24]:
removeEmptyFields(bc.loc[bc['Mounting Arrangement'].isna()==False])['Mounting Arrangement'].unique()

array(['Triplex-Universal', 'Quadruplex-Back to Back'], dtype=object)

In [25]:
removeEmptyFields(bc.loc[bc['Bearing Arrangement'].isna()==False])['Bearing Arrangement'].unique()

array(['Universal', 'Duplex-Universal', 'Duplex-Back to Back',
       'Triplex-Tandem and Back to Back',
       'Quadruplex-Tandem and Face to Face', 'Duplex-Face to Face',
       'Duplex-Tandem', 'Quadruplex-Tandem and Back to Back',
       'Triplex-Universal', 'Triplex-Tandem and Face to Face',
       'Quadruplex-Universal', 'Quadruplex-Back to Back', 'Steel',
       'Triplex-Tandem'], dtype=object)

In [26]:
removeEmptyFields(bc.loc[bc['Mounting Arrangement'].isna()==False])[['Mounting Arrangement', 'Bearing Arrangement']]

,Mounting Arrangement,Bearing Arrangement
53721,Triplex-Universal,Triplex-Universal
57373,Quadruplex-Back to Back,Quadruplex-Back to Back
